In [18]:
import pandas as pd
from skyfield.api import N, E, wgs84, load
import datetime as dt

# Data inladen
df = pd.read_csv('data/raw-gyro.csv')
df.head()

# FUNCTIES ---------------------------------------------------------------------------------------------------------------
# Functie om UTC naar CET of CEST te converteren op basis van de datum
def utc_to_cet_or_cest(utc_dt):
    # Definieer de tijdzones
    utc_zone = pytz.utc
    cet_zone = pytz.timezone('Europe/Amsterdam')
    
    # Zet de datetime naar een aware datetime in UTC
    utc_dt = utc_zone.localize(utc_dt)
    
    # Converteer naar CET of CEST
    cet_or_cest_dt = utc_dt.astimezone(cet_zone)
    
    return cet_or_cest_dt

def datetime_to_tuple(dt):
    return (dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)

# Functie om de azimut te berekenen
def calculate_azimuth(time_tuple):
    year, month, day, hour, minute, second = time_tuple
    t = ts.utc(year, month, day, hour, minute, second)
    astro = Nieuwe_diep.at(t).observe(sun)
    app = astro.apparent()
    _, az, _ = app.altaz()
    return az.degrees

# Functie om de juiste kolomnaam te bepalen en de waarde toe te voegen
def add_localtime_column(row):
    cet_or_cest_dt = utc_to_cet_or_cest(row['datetime_UTC'])
    if cet_or_cest_dt.tzname() == 'CEST':
        row['localtime_CEST'] = cet_or_cest_dt
    else:
        row['localtime_CET'] = cet_or_cest_dt
    return row

# Functie om de azimut te berekennen ------------------------------------------------------------------------------------------------------
def calculate_azimuth(time_tuple):
    year, month, day, hour, minute, second = time_tuple
    t = ts.utc(year, month, day, hour, minute, second)
    astro = Nieuwe_diep.at(t).observe(sun)
    app = astro.apparent()
    _, az, _ = app.altaz()
    return az.degrees

# Azimut berekenen voor elke rij in de DataFrame
# df['azimuth'] = df['time_tuple'].apply(calculate_azimuth)

# Functie om de juiste kolomnaam te bepalen en de waarde toe te voegen
def add_localtime_column(row):
    cet_or_cest_dt = utc_to_cet_or_cest(row['datetime_UTC'])
    if cet_or_cest_dt.tzname() == 'CEST':
        row['localtime_CEST'] = cet_or_cest_dt
    else:
        row['localtime_CET'] = cet_or_cest_dt
    return row

# Locatie Port of Den Helder, Nieuwe diep:
lat = 52 + (57 / 60) + (26.9 / 3600)
lon = 4 + (46 / 60) + (37.5 / 3600)
height_m = 6

# Skyfield setup
ts = load.timescale()
planets = load('de421.bsp')
earth, sun = planets['earth'], planets['sun']
Nieuwe_diep = earth + wgs84.latlon(lat * N, lon * E, elevation_m=height_m)

df['time_tuple'] = df['datetime_UTC'].apply(datetime_to_tuple)





In [19]:
import pandas as pd
from skyfield.api import load, wgs84
from skyfield.api import N, E
import pytz

# Example DataFrame
data = {
    'UTC': ['31-12-2019 08:41:44', '31-12-2019 08:43:16', '31-12-2019 08:44:12', '31-12-2019 08:44:52'],
    'obs_sun_deg': [343, 344, 344, 344],
    'obs_sun_min': [41, 6, 16, 22],
    'obs_sun_sec': [24, 0, 44, 5],
    'obs_gyro_deg_true': [155.063, 154.985, 155.055, 155.058]
}
df = pd.DataFrame(data)

# Location Port of Den Helder, Nieuwe diep:
lat = 52 + (57 / 60) + (26.9 / 3600)
lon = 4 + (46 / 60) + (37.5 / 3600)
height_m = 6

# Skyfield setup
ts = load.timescale()
planets = load('de421.bsp')
earth, sun = planets['earth'], planets['sun']
Nieuwe_diep = earth + wgs84.latlon(lat * N, lon * E, elevation_m=height_m)

# Function to calculate azimuth
def calculate_azimuth(utc_time):
    t = ts.utc(utc_time.year, utc_time.month, utc_time.day, utc_time.hour, utc_time.minute, utc_time.second)
    astro = Nieuwe_diep.at(t).observe(sun)
    app = astro.apparent()
    _, az, _ = app.altaz()
    return az.degrees

# Convert the UTC column to datetime objects with UTC timezone
df['UTC'] = pd.to_datetime(df['UTC'], format='%d-%m-%Y %H:%M:%S').dt.tz_localize(pytz.UTC)

# Calculate azimuth for each row in the DataFrame and round to three decimal places
df['azimuth'] = df['UTC'].apply(calculate_azimuth).round(3)

# Convert UTC datetime objects to string with Zulu time format
df['UTC'] = df['UTC'].dt.strftime('%d-%m-%Y %H:%M:%S%z')

# Drop the original DMS columns
df.drop(['obs_sun_deg', 'obs_sun_min', 'obs_sun_sec'], axis=1, inplace=True)

# Display the result
print(df)

                        UTC  obs_gyro_deg_true  azimuth
0  31-12-2019 08:41:44+0000            155.063  138.873
1  31-12-2019 08:43:16+0000            154.985  139.192
2  31-12-2019 08:44:12+0000            155.055  139.387
3  31-12-2019 08:44:52+0000            155.058  139.526


The DataFrame object before deleting the column
   a  b
0  1  2
1  2  4
2  3  6
The DataFrame object after deleting the column a
   b
0  2
1  4
2  6


In [1]:
pos= 
raw = 'gyro_sunshot_31-12-19.csv'
dfr = pd.read_csv(raw, names=['fix_nr','time','obs_sun_D','obs_sun_M','obs_sun_S','d0','d1','d2','obs_gyro_true','d3','d4']) 
df = dfr.drop(['d0','d1','d2','d3','d4'], axis=1) # d0/d4 will be removed

df.head()

,fix_nr,time,obs_sun_D,obs_sun_M,obs_sun_S,obs_gyro_true
0,1,08:41:44,343,41,24,155.063
1,2,08:43:16,344,6,0,154.985
2,3,08:44:12,344,16,44,155.055
3,4,08:44:52,344,22,5,155.058
4,5,08:46:01,344,35,30,155.076


In [5]:
# Create separate dataframe for DMS to decimal operations & datetime column
d1 = {'fix_nr':  dfr['fix_nr'], 
      'obs_sun_D': dfr['obs_sun_D'],
      'obs_sun_M': dfr['obs_sun_M'],
      'obs_sun_S': dfr['obs_sun_S']
    }
df1 = pd.DataFrame( data=d1 ) # dms 2 deg.deg

df1["obs_sun_decimal"] = df1[["obs_sun_D","obs_sun_M","obs_sun_S"]].apply(lambda row: row.values[0] + row.values[1]/60 + row.values[2]/3600, axis=1)
df1.head()

 
df['obs_sun_decimal'] = df1['obs_sun_decimal']

# adding a date to the time in dfr
d2 = {'date': ['2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31',
             '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31',
             '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31',
             '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31']
     }
df2= pd.DataFrame(data=d2)
df2['time'] = dfr.time # default time is UTC
df2['datetime_UTC'] = pd.to_datetime(df2['date'].astype(str) + ' ' + df2['time'].astype(str))

"""
DF TO DATETIME w/ Pandas:
https://stackoverflow.com/questions/53470304/convert-time-object-to-datetime-format-in-python-pandas

print(pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str)))
"""
df2

# adding additional columns to df 
df['obs_sun_decimal'] = df1['obs_sun_decimal']
df['datetime_UTC']= df2['datetime_UTC']
col_order = ['fix_nr','datetime_UTC','obs_sun_D','obs_sun_M','obs_sun_S','obs_sun_decimal','obs_gyro_true'] 
df.drop(['time'], axis=1)
df = df.reindex(columns=col_order)
df

,fix_nr,datetime_UTC,obs_sun_D,obs_sun_M,obs_sun_S,obs_sun_decimal,obs_gyro_true
0,1,2019-12-31 08:41:44,343,41,24,343.690000,155.063
1,2,2019-12-31 08:43:16,344,6,0,344.100000,154.985
2,3,2019-12-31 08:44:12,344,16,44,344.278889,155.055
3,4,2019-12-31 08:44:52,344,22,5,344.368056,155.058
4,5,2019-12-31 08:46:01,344,35,30,344.591667,155.076
...,...,...,...,...,...,...,...
15,16,2019-12-31 09:08:50,349,42,35,349.709722,154.768
16,17,2019-12-31 09:09:13,349,51,28,349.857778,154.733
17,18,2019-12-31 09:09:33,350,3,0,350.050000,154.597
18,19,2019-12-31 09:09:57,350,16,46,350.279444,154.509


In [12]:
from skyfield.api import N,S,E,W, wgs84, load

ts = load.timescale()
t = ts.utc(2019, 12, 31)
planets = load('de421.bsp')
earth, sun = 
# Altitude and azimuth in the sky of a
# specific geographic location
earth = planets['earth']
Nieuwe_diep = earth + wgs84.latlon(42.3583 * N, 71.0603 * W, elevation_m=43)
astro = boston.at(ts.utc(1980, 3, 1)).observe(mars)
app = astro.apparent()

alt, az, distance = app.altaz()
print(alt.dstr())
print(az.dstr())
print(distance)

SyntaxError: invalid syntax (3227825788.py, line 7)

In [11]:
from skyfield.api import load

ts = load.timescale()
t = ts.utc(2019, 12, 31)
planets = load('de421.bsp')
earth, mars = planets['earth'], planets['mars']

# Three positions in a single line of code!
d = earth.at(t).observe(mars).apparent().distance()

print('Mars is {:.2f} au from Earth'.format(d.au))

Mars is 2.19 au from Earth


In [10]:
print(planets)

SPICE kernel file 'de421.bsp' has 15 segments
  JD 2414864.50 - JD 2471184.50  (1899-07-28 through 2053-10-08)
      0 -> 1    SOLAR SYSTEM BARYCENTER -> MERCURY BARYCENTER
      0 -> 2    SOLAR SYSTEM BARYCENTER -> VENUS BARYCENTER
      0 -> 3    SOLAR SYSTEM BARYCENTER -> EARTH BARYCENTER
      0 -> 4    SOLAR SYSTEM BARYCENTER -> MARS BARYCENTER
      0 -> 5    SOLAR SYSTEM BARYCENTER -> JUPITER BARYCENTER
      0 -> 6    SOLAR SYSTEM BARYCENTER -> SATURN BARYCENTER
      0 -> 7    SOLAR SYSTEM BARYCENTER -> URANUS BARYCENTER
      0 -> 8    SOLAR SYSTEM BARYCENTER -> NEPTUNE BARYCENTER
      0 -> 9    SOLAR SYSTEM BARYCENTER -> PLUTO BARYCENTER
      0 -> 10   SOLAR SYSTEM BARYCENTER -> SUN
      3 -> 301  EARTH BARYCENTER -> MOON
      3 -> 399  EARTH BARYCENTER -> EARTH
      1 -> 199  MERCURY BARYCENTER -> MERCURY
      2 -> 299  VENUS BARYCENTER -> VENUS
      4 -> 499  MARS BARYCENTER -> MARS
